# 1. Filtrage

## 1.1. Base de donnees STRING

In [29]:
import pandas as pd
from pathlib import Path

# Chemins des fichiers
input_path = Path(r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\raw data\Protein_Interactions\STRING_Interactions.txt")
output_path = Path(r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\interactions\STRING_filtered_interactions.txt")

# 1. Chargement et filtrage initial
df = pd.read_csv(input_path, sep=" ")

# Créer un masque de filtrage
mask = (df["combined_score"] >= 900) & (
    df[["experimental", "coexpression", "database", "textmining"]].max(axis=1) > 90
)

# Appliquer le filtre et faire une copie explicite
filtered_df = df.loc[mask].copy()

# 2. Nettoyage des données
# Supprimer "4932." des noms de protéines
for col in ['protein1', 'protein2']:
    filtered_df.loc[:, col] = filtered_df[col].str.replace('4932.', '')

# 3. Gestion des interactions uniques
# Créer des identifiants d'interaction canoniques
filtered_df.loc[:, 'interaction_key'] = filtered_df.apply(
    lambda x: frozenset({x['protein1'], x['protein2']}), axis=1
)

# Supprimer les doublons en gardant la première occurrence
unique_interactions_df = filtered_df.drop_duplicates(subset='interaction_key')

# 4. Sauvegarde des résultats
unique_interactions_df[['protein1', 'protein2']].to_csv(
    output_path, sep="\t", index=False, header=False
)

# 5. Calcul des statistiques
unique_proteins = pd.unique(
    unique_interactions_df[['protein1', 'protein2']].values.ravel('K')
)
num_interactions = len(unique_interactions_df)

# Affichage des résultats
print("\n Traitement (filtrage) terminé avec succès.")
print(f"  - Nombre de protéines uniques : {len(unique_proteins)}")
print(f"  - Nombre d'interactions uniques : {num_interactions}")


 Traitement (filtrage) terminé avec succès.
  - Nombre de protéines uniques : 4708
  - Nombre d'interactions uniques : 53150


## 1.2. Base de donnees DIP

In [23]:
import xml.etree.ElementTree as ET
from pathlib import Path

def process_dip_interactions():
    # Configuration des chemins
    input_file = Path(r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\raw data\Protein_Interactions\DIP_Interactions.mif25")
    output_file = Path(r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\interactions\DIP_filtered_interactions.txt")
    
    NS = {'mif': 'http://psi.hupo.org/mi/mif'}
    TARGET_TAXID = "4932"  # TaxID de Saccharomyces cerevisiae
    
    tree = ET.parse(input_file)
    root = tree.getroot()

    # 1. Extraction des protéines avec vérification de l'organisme
    id_to_protein = {}
    for interactor in root.findall(".//mif:interactor", NS):
        interactor_id = interactor.get("id")
        if not interactor_id:
            continue
            
        # Vérification que la protéine appartient à la levure
        organism = interactor.find(".//mif:organism", NS)
        if organism is None or organism.get("ncbiTaxId") != TARGET_TAXID:
            continue
            
        uniprot_ref = interactor.find(".//mif:xref/mif:secondaryRef[@db='uniprot knowledge base']", NS)
        refseq_ref = interactor.find(".//mif:xref/mif:secondaryRef[@db='refseq']", NS)
        
        protein_id = None
        if uniprot_ref is not None:
            protein_id = uniprot_ref.get("id")
        elif refseq_ref is not None:
            protein_id = refseq_ref.get("id")
        else:
            short_label = interactor.find(".//mif:names/mif:shortLabel", NS)
            protein_id = short_label.text if short_label is not None else None
        
        if protein_id:
            id_to_protein[interactor_id] = protein_id

    # 2. Extraction des interactions avec contrôle qualité
    unique_interactions = set()
    protein_set = set()  # Pour stocker les protéines uniques
    
    for interaction in root.findall(".//mif:interaction", NS):
        participants = interaction.findall(".//mif:participant/mif:interactorRef", NS)
        
        if len(participants) != 2:
            continue
            
        id1, id2 = participants[0].text, participants[1].text
        
        # Vérification que les deux protéines existent, sont différentes et appartiennent à la levure
        if (id1 not in id_to_protein or 
            id2 not in id_to_protein or 
            id_to_protein[id1] == id_to_protein[id2]):
            continue
            
        prot1, prot2 = id_to_protein[id1], id_to_protein[id2]
        
        # Ajout aux protéines uniques
        protein_set.add(prot1)
        protein_set.add(prot2)
        
        # Vérification du score
        score_element = interaction.find(".//mif:confidence/mif:value", NS)
        if score_element is not None:
            try:
                score = float(score_element.text)
                if score <= 0.8:
                    continue
            except (ValueError, TypeError):
                continue
                
        # Ajout sous forme triée pour éviter les doublons A-B vs B-A
        sorted_interaction = tuple(sorted((prot1, prot2)))
        unique_interactions.add(sorted_interaction)

    # 3. Sauvegarde
    with open(output_file, "w") as f:
        f.write("Protein1\tProtein2\n")
        for prot1, prot2 in unique_interactions:
            f.write(f"{prot1}\t{prot2}\n")

    # 4. Calcul et affichage des statistiques
    num_unique_proteins = len(protein_set)
    num_unique_interactions = len(unique_interactions)
    
    print("\nRésultats du traitement (Saccharomyces cerevisiae uniquement):")
    print(f"- Protéines uniques: {num_unique_proteins}")
    print(f"- Interactions uniques: {num_unique_interactions}")
    print(f"Fichier sauvegardé: {output_file}")

if __name__ == "__main__":
    process_dip_interactions()


Résultats du traitement (Saccharomyces cerevisiae uniquement):
- Protéines uniques: 5012
- Interactions uniques: 22436
Fichier sauvegardé: C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\interactions\DIP_filtered_interactions.txt


## 1.3. Base de donnes BIOGRID

In [24]:
import pandas as pd
import re
from pathlib import Path

def process_biogrid_high_confidence():
    # Chemins des fichiers
    input_file = Path(r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\raw data\Protein_Interactions\BIOGRID-ORGANISM-Saccharomyces_cerevisiae.txt")
    output_file = Path(r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\interactions\BIOGRID_top_40000.tsv")
    
    # Charger les données
    df = pd.read_csv(input_file, sep='\t', comment='#', header=None)
    df.columns = [
        'ID_A', 'ID_B', 'Alt_IDs_A', 'Alt_IDs_B', 
        'Aliases_A', 'Aliases_B', 'Method', 'Author',
        'PubIDs', 'TaxID_A', 'TaxID_B', 'IntType',
        'SourceDB', 'IntIDs', 'Confidence'
    ]
    
    # 1. Filtrer pour la levure S288c
    yeast_df = df[(df['TaxID_A'] == 'taxid:559292') & 
                 (df['TaxID_B'] == 'taxid:559292')].copy()
    
    # 2. Critères de haute confiance
    high_conf_methods = [
        'affinity chromatography',
        'two hybrid',
        'pull down',
        'coimmunoprecipitation'
    ]
    
    high_conf_types = [
        'direct interaction',
        'physical association'
    ]
    
    # 3. Filtrer avec critères combinés
    method_mask = yeast_df['Method'].str.contains('|'.join(high_conf_methods), case=False, na=False)
    type_mask = yeast_df['IntType'].str.contains('|'.join(high_conf_types), case=False, na=False)
    
    high_conf = yeast_df[method_mask & type_mask].copy()
    
    # 4. Extraire les UniProt IDs
    def extract_uniprot(alt_ids):
        if pd.isna(alt_ids): return None
        match = re.search(r'uniprot/swiss[\W-]?prot:([A-Z0-9]{6,10})', str(alt_ids))
        return match.group(1) if match else None
    
    high_conf.loc[:, 'Protein1'] = high_conf['Alt_IDs_A'].apply(extract_uniprot)
    high_conf.loc[:, 'Protein2'] = high_conf['Alt_IDs_B'].apply(extract_uniprot)
    
    # 5. Nettoyage final
    clean_df = high_conf.dropna(subset=['Protein1', 'Protein2'])
    clean_df = clean_df[clean_df['Protein1'] != clean_df['Protein2']]
    
    # 6. Sélection des 40 000 interactions les plus fréquentes
    # Compter les occurrences de chaque interaction
    interaction_counts = pd.concat([
        clean_df.groupby('Protein1').size().rename('Count'),
        clean_df.groupby('Protein2').size().rename('Count')
    ], axis=1).fillna(0)
    interaction_counts['Total'] = interaction_counts.sum(axis=1)
    
    # Ajouter les scores aux interactions
    clean_df = clean_df.merge(
        interaction_counts[['Total']].rename(columns={'Total': 'Score1'}),
        left_on='Protein1', right_index=True
    ).merge(
        interaction_counts[['Total']].rename(columns={'Total': 'Score2'}),
        left_on='Protein2', right_index=True
    )
    clean_df['InteractionScore'] = clean_df['Score1'] + clean_df['Score2']
    
    # Trier et sélectionner le top 40 000
    top_interactions = clean_df.sort_values('InteractionScore', ascending=False)\
                             .drop_duplicates(subset=['Protein1', 'Protein2'])\
                             .head(40000)
    
    # 7. Statistiques
    unique_proteins = pd.unique(top_interactions[['Protein1', 'Protein2']].values.ravel('K'))
    
    print("\nRésultats sélection:")
    print(f"- Interactions totales (levure): {len(yeast_df)}")
    print(f"- Interactions haute confiance: {len(high_conf)}")
    print(f"- Interactions sélectionnées: {len(top_interactions)}")
    print(f"- Protéines uniques: {len(unique_proteins)}")
    
    # 8. Sauvegarde
    top_interactions[['Protein1', 'Protein2']].to_csv(
        output_file, 
        sep='\t', 
        index=False, 
        header=False
    )
    print(f"\nTop 40k interactions sauvegardées dans: {output_file}")

if __name__ == "__main__":
    process_biogrid_high_confidence()


Résultats sélection:
- Interactions totales (levure): 855577
- Interactions haute confiance: 224490
- Interactions sélectionnées: 40000
- Protéines uniques: 5519

Top 40k interactions sauvegardées dans: C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\interactions\BIOGRID_top_40000.tsv


## 1.4. Base de donnees (Complex Portal)

In [25]:
import os
import zipfile
import shutil
from xml.etree import ElementTree as ET

# Chemins
zip_path = r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\raw data\Complexes\yeast.zip"
extract_folder = r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\raw data\Complexes\yeast_extracted"
output_file = r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\complexes\complexes_levure.txt"

# 1. Nettoyage et extraction
if os.path.exists(extract_folder):
    shutil.rmtree(extract_folder)
os.makedirs(extract_folder, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

# 2. Traitement spécifique pour votre format XML
complexes = []
yeast_folder = os.path.join(extract_folder, "yeast")

for filename in os.listdir(yeast_folder):
    if filename.endswith(".xml"):
        filepath = os.path.join(yeast_folder, filename)
        
        try:
            tree = ET.parse(filepath)
            root = tree.getroot()
            
            # Namespace spécifique à vos fichiers
            ns = {'mif': 'http://psi.hupo.org/mi/mif300'}
            
            # Recherche des interactions complexes
            for interaction in root.findall(".//mif:abstractInteraction", ns):
                proteins = set()
                
                # Recherche des participants
                for participant in interaction.findall(".//mif:participant", ns):
                    # Référence à l'interacteur
                    interactor_ref = participant.find(".//mif:interactorRef", ns)
                    if interactor_ref is not None:
                        # Trouver l'interacteur correspondant
                        interactor = root.find(f".//mif:interactor[@id='{interactor_ref.text}']", ns)
                        if interactor is not None:
                            # Vérifier si c'est une protéine
                            interactor_type = interactor.find(".//mif:interactorType/mif:names/mif:shortLabel", ns)
                            if interactor_type is not None and interactor_type.text == "protein":
                                # Récupérer l'identifiant UniProt
                                uniprot = interactor.find(".//mif:xref/mif:primaryRef[@db='uniprotkb']", ns)
                                if uniprot is not None:
                                    proteins.add(uniprot.get("id"))
                
                if proteins:
                    complexes.append(sorted(proteins))
        
        except Exception as e:
            print(f"Erreur avec {filename}: {str(e)[:200]}")

# 3. Écriture du fichier final
with open(output_file, 'w', encoding='utf-8') as f_out:
    # Format: ID [tab] Liste_de_protéines (séparées par des espaces)
    for idx, proteins in enumerate(complexes, 1):
        f_out.write(f"{idx}\t{' '.join(proteins)}\n")

# 4. Rapport
print(f"Fichier généré: {output_file}")
print(f"Nombre total de complexes trouvés: {len(complexes)}")


Fichier généré: C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\complexes\complexes_levure.txt
Nombre total de complexes trouvés: 643


## 1.5. filtrage de complexes

In [28]:
import os
from collections import defaultdict

# Chemins des fichiers
complexes_file = r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\complexes\complexes_levure.txt"

reseau_files = [
    r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\interactions\STRING_filtered_interactions.txt",
    r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\interactions\DIP_filtered_interactions.txt",
    r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\interactions\BIOGRID_top_40000.tsv"
]
output_files = [
    r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\complexes\complexes_STRING.txt",
    r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\complexes\complexes_DIP.txt",
    r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\complexes\complexes_BIOGRID.txt"
]

# 1. Charger les complexes depuis complexes_levure.txt
def load_complexes(file_path):
    complexes = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) >= 2:
                proteins = set(parts[1].split())
                complexes.append(proteins)
    return complexes

complexes = load_complexes(complexes_file)
print(f"Nombre total de complexes chargés: {len(complexes)}")

# 2. Charger un réseau PPI depuis un fichier (gère les TSV/CSV et ignore les en-têtes)
def load_ppi_network(file_path):
    network = defaultdict(set)
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#") or line.startswith("protein1"):  # Ignorer lignes vides/en-têtes
                continue
            parts = line.split('\t') if file_path.endswith('.tsv') else line.split()
            if len(parts) >= 2:
                prot1, prot2 = parts[0], parts[1]  # Prendre les 2 premières colonnes
                network[prot1].add(prot2)
                network[prot2].add(prot1)
    return network

# 3. Vérifier si un complexe est entièrement couvert par le réseau
def is_complex_covered(complexe, network):
    proteins = list(complexe)
    for i in range(len(proteins)):
        for j in range(i + 1, len(proteins)):
            prot1, prot2 = proteins[i], proteins[j]
            if prot2 not in network.get(prot1, set()):
                return False
    return True

# 4. Traiter chaque réseau PPI et sauvegarder les résultats
for reseau_file, output_file in zip(reseau_files, output_files):
    print(f"\nTraitement de {reseau_file}...")
    try:
        network = load_ppi_network(reseau_file)
        
        # Trouver les complexes couverts
        covered_complexes = []
        for complexe in complexes:
            if is_complex_covered(complexe, network):
                covered_complexes.append(complexe)
        
        # Sauvegarder dans le fichier de sortie
        with open(output_file, 'w', encoding='utf-8') as f_out:
            for idx, complexe in enumerate(covered_complexes, 1):
                f_out.write(f"{idx}\t{' '.join(complexe)}\n")
        
        print(f"→ {len(covered_complexes)} complexes trouvés dans ce réseau.")
        print(f"Fichier généré: {output_file}")
    except Exception as e:
        print(f"Erreur avec {reseau_file}: {str(e)}")

print("\nTerminé avec succès !")

Nombre total de complexes chargés: 643

Traitement de C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\interactions\STRING_filtered_interactions.txt...
→ 21 complexes trouvés dans ce réseau.
Fichier généré: C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\complexes\complexes_STRING.txt

Traitement de C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\interactions\DIP_filtered_interactions.txt...
→ 278 complexes trouvés dans ce réseau.
Fichier généré: C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\complexes\complexes_DIP.txt

Traitement de C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\interactions\BIOGRID_top_40000.tsv...
→ 22 complexes trouvés dans ce réseau.
Fichier généré: C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\complexes\complexes_BIOGRID.txt

Terminé avec succès !
